In [4]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


----
---
---
---
---
---


# 93.01 % ACCURACY CODE.
## Total params: 995,230

[My Densenet Blog ](https://karthikziffer.github.io/journal/summary-densenet.html)
---
---
---


1 . Batch Size - - 128 - This batch size reached a test accuracy of 91% accuracy, but couldnot converge beyond 92%. The result remained the same for both data augmentation and without data augmentation. - 64 - By reducing the batch size, the model had much room to update the weights during back propagation. This resulted in a test accuracy of 93%. - 34 - For this batch size, the model training time was higher, but there was no much improvement in test accuracy. 


2 . Train data normalization and data augmentation - - Since CIFAR10 dataset, consists of images shot at different lighting condition and angle. If the images are at differenet resolution level, Normalizing it would scale the entire dataset images accordingly. - From DENSENET paper, the authors have considered two data augmentation tehnique.
1. Increase the width and height of the image and crop it randomly. 
2. Horizontal shifting of images. I considered, width shift range and height shift range along with horizontal shift for 180 epochs.This helped me to reach an test accuracy of 92%. Then I removed data augmentation, for another 70 epochs, this increased both my training accuracy and test accuracy to 93%.


**Conclusion**: By training the model with data augmentation for 200 epochs, and then removing augmentation will increase the test accuracy considerably. 1. Learning Rate scheduler - By refering to the paper, for initial 150 epochs the lr was 0.1 , next till 225 the lr was 0.01 and for ramaining epochs it was 0.001. ''" Learning rate annealing recommends starting with a relatively high learning rate and then gradually lowering the learning rate during training. The intuition behind this approach is that we'd lika a learning rate small enough that we can explore the deeper but narrower parts of the loss function"'' - By Andrej Karparthy.

# Oct 22 
---

# 1 - 55 Epoch

## Validation Accuracy - 85%

In [2]:

#keras library

import keras
import keras.backend as K
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam 
from keras.layers import Input
from keras.callbacks import LearningRateScheduler


# densenet 

def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp
  
  
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg
 




def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output
  
#############################################################################
# Hyperparameters
batch_size = 64
num_classes = 10
epochs = 55
l = 12 #L
num_filter = 36 #k
compression = 0.5
dropout_rate = 0.2
#############################################################################
  
  
  
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]




# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)



#normalizing data

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255





input = Input(shape=(img_height, img_width, channel))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

# Back_Prop_First_Conv2D

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)


Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)



model = Model(inputs=[input], outputs=[output])
# model.summary()



#optimizer
sgd = keras.optimizers.SGD(lr=0.1 ,  momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

#callbacks


def lr_schedule(epoch):
  if epoch <= 150:
    K.set_value(model.optimizer.lr, 0.1)
    print("learning rate --->" , 0.1 , "Epoch --->" , epoch)
    return K.get_value(model.optimizer.lr)
  elif epoch > 150 and epoch <=225:
    K.set_value(model.optimizer.lr, 0.01)
    print("learning rate --->" , 0.01 , "Epoch --->" , epoch)
    return K.get_value(model.optimizer.lr)
  else:
    K.set_value(model.optimizer.lr, 0.001)
    print("learning rate --->" , 0.001 , "Epoch --->" , epoch)
    return K.get_value(model.optimizer.lr)
    
      
  
lr_reducer = LearningRateScheduler(lr_schedule)

filepath = '/gdrive/My Drive/CIFAR_10/weights.{epoch:02d}-{val_acc:.2f}.hdf5'
model_checkpoint = keras.callbacks.ModelCheckpoint(filepath,
                                monitor='val_acc', 
                                verbose=1, 
                                save_best_only=True, 
                                save_weights_only=False, 
                                mode='max', 
                                period=5)


csv_logger = keras.callbacks.CSVLogger(filename =  "fifth__.csv", separator=',', append=False)


datagen = keras.preprocessing.image.ImageDataGenerator(
#     featurewise_std_normalization=True,
#     rotation_range=9 , 
    width_shift_range=0.125,
    height_shift_range=0.125,
    horizontal_flip=True)



datagen.fit(x_train)
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    callbacks=[lr_reducer , csv_logger , model_checkpoint] ,
#                     steps_per_epoch= len(x_train)/batch_size,
                    epochs=epochs , 
                   verbose = 1 , 
                   workers = 4 , 
#                    use_multiprocessing = True , 
                   shuffle = True , 
                   validation_data=(x_test, y_test))



Using TensorFlow backend.


170500096/170498071 [==============================] - 86s 1us/step
Epoch 1/55
learning rate ---> 0.1 Epoch ---> 0
782/782 [==============================] - 311s 398ms/step - loss: 2.3500 - acc: 0.2070 - val_loss: 1.8722 - val_acc: 0.3054
Epoch 2/55
learning rate ---> 0.1 Epoch ---> 1
782/782 [==============================] - 296s 378ms/step - loss: 1.7681 - acc: 0.3392 - val_loss: 1.7469 - val_acc: 0.3362
Epoch 3/55
learning rate ---> 0.1 Epoch ---> 2
782/782 [==============================] - 296s 379ms/step - loss: 1.5673 - acc: 0.4217 - val_loss: 1.5570 - val_acc: 0.4376
Epoch 4/55
learning rate ---> 0.1 Epoch ---> 3
782/782 [==============================] - 296s 379ms/step - loss: 1.3899 - acc: 0.4923 - val_loss: 1.6115 - val_acc: 0.4590
Epoch 5/55
learning rate ---> 0.1 Epoch ---> 4
782/782 [==============================] - 298s 381ms/step - loss: 1.2633 - acc: 0.5453 - val_loss: 1.4195 - val_acc: 0.5025

Epoch 00005: val_acc improved from -inf to 0.50250, saving model to /gd

# 56 - 100 Epoch

## Validation Accuracy - 89%

In [5]:
model.load_weights("/gdrive/My Drive/CIFAR_10/weights.55-0.87.hdf5")

sgd = keras.optimizers.SGD(lr=0.1 ,  momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

#callbacks


def lr_schedule(epoch):
  
  epoch += 55 
  if epoch <= 150:
    K.set_value(model.optimizer.lr, 0.1)
    print("learning rate --->" , 0.1 , "Epoch --->" , epoch)
    return K.get_value(model.optimizer.lr)
  elif epoch > 150 and epoch <=225:
    K.set_value(model.optimizer.lr, 0.01)
    print("learning rate --->" , 0.01 , "Epoch --->" , epoch)
    return K.get_value(model.optimizer.lr)
  else:
    K.set_value(model.optimizer.lr, 0.001)
    print("learning rate --->" , 0.001 , "Epoch --->" , epoch)
    return K.get_value(model.optimizer.lr)
    
      
  
lr_reducer = LearningRateScheduler(lr_schedule)

filepath = '/gdrive/My Drive/CIFAR_10/weights.{epoch:02d}-{val_acc:.2f}.hdf5'
model_checkpoint = keras.callbacks.ModelCheckpoint(filepath,
                                monitor='val_acc', 
                                verbose=1, 
                                save_best_only=True, 
                                save_weights_only=False, 
                                mode='max', 
                                period=5)


csv_logger = keras.callbacks.CSVLogger(filename =  "fifth__.csv", separator=',', append=False)


datagen = keras.preprocessing.image.ImageDataGenerator(
#     featurewise_std_normalization=True,
#     rotation_range=9 , 
    width_shift_range=0.125,
    height_shift_range=0.125,
    horizontal_flip=True)



datagen.fit(x_train)

model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    callbacks=[lr_reducer , csv_logger , model_checkpoint] ,
#                     steps_per_epoch= len(x_train)/batch_size,
                    epochs=45 , 
                   verbose = 1 , 
                   workers = 4 , 
#                    use_multiprocessing = True , 
                   shuffle = True , 
                   validation_data=(x_test, y_test))

Epoch 1/45
learning rate ---> 0.1 Epoch ---> 55
782/782 [==============================] - 309s 395ms/step - loss: 0.3191 - acc: 0.8897 - val_loss: 0.4737 - val_acc: 0.8688
Epoch 2/45
learning rate ---> 0.1 Epoch ---> 56
782/782 [==============================] - 290s 370ms/step - loss: 0.3109 - acc: 0.8910 - val_loss: 0.5782 - val_acc: 0.8369
Epoch 3/45
learning rate ---> 0.1 Epoch ---> 57
782/782 [==============================] - 284s 363ms/step - loss: 0.3138 - acc: 0.8884 - val_loss: 0.4684 - val_acc: 0.8673
Epoch 4/45
learning rate ---> 0.1 Epoch ---> 58
782/782 [==============================] - 290s 370ms/step - loss: 0.3097 - acc: 0.8931 - val_loss: 0.4202 - val_acc: 0.8778
Epoch 5/45
learning rate ---> 0.1 Epoch ---> 59
782/782 [==============================] - 289s 370ms/step - loss: 0.3023 - acc: 0.8954 - val_loss: 0.5740 - val_acc: 0.8486

Epoch 00005: val_acc improved from -inf to 0.84860, saving model to /gdrive/My Drive/CIFAR_10/weights.05-0.85.hdf5
Epoch 6/45
learning

# 100 - 150 Epoch

## Validation Accuracy - 91.99%

In [8]:
model.load_weights("/gdrive/My Drive/CIFAR_10/weights.15-0.89.hdf5")

sgd = keras.optimizers.SGD(lr=0.1 ,  momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

#callbacks


def lr_schedule(epoch):
  
  epoch += 100 
  if epoch <= 120:
    K.set_value(model.optimizer.lr, 0.1)
    print("learning rate --->" , 0.1 , "Epoch --->" , epoch)
    return K.get_value(model.optimizer.lr)
  elif epoch > 120 and epoch <=210:
    K.set_value(model.optimizer.lr, 0.01)
    print("learning rate --->" , 0.01 , "Epoch --->" , epoch)
    return K.get_value(model.optimizer.lr)
  else:
    K.set_value(model.optimizer.lr, 0.001)
    print("learning rate --->" , 0.001 , "Epoch --->" , epoch)
    return K.get_value(model.optimizer.lr)
    
      
  
lr_reducer = LearningRateScheduler(lr_schedule)

filepath = '/gdrive/My Drive/CIFAR_10/weights.{epoch:02d}-{val_acc:.2f}.hdf5'
model_checkpoint = keras.callbacks.ModelCheckpoint(filepath,
                                monitor='val_acc', 
                                verbose=1, 
                                save_best_only=True, 
                                save_weights_only=False, 
                                mode='max', 
                                period=1)


csv_logger = keras.callbacks.CSVLogger(filename =  "fifth__.csv", separator=',', append=False)


datagen = keras.preprocessing.image.ImageDataGenerator(
#     featurewise_std_normalization=True,
#     rotation_range=9 , 
    width_shift_range=0.125,
    height_shift_range=0.125,
    horizontal_flip=True)



datagen.fit(x_train)

model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    callbacks=[lr_reducer , csv_logger , model_checkpoint] ,
#                     steps_per_epoch= len(x_train)/batch_size,
                    epochs=45 , 
                   verbose = 1 , 
                   workers = 4 , 
#                    use_multiprocessing = True , 
                   shuffle = True , 
                   validation_data=(x_test, y_test))

Epoch 1/45
learning rate ---> 0.1 Epoch ---> 100
782/782 [==============================] - 311s 398ms/step - loss: 0.2192 - acc: 0.9221 - val_loss: 0.3970 - val_acc: 0.8907

Epoch 00001: val_acc improved from -inf to 0.89070, saving model to /gdrive/My Drive/CIFAR_10/weights.01-0.89.hdf5
Epoch 2/45
learning rate ---> 0.1 Epoch ---> 101
782/782 [==============================] - 297s 380ms/step - loss: 0.2178 - acc: 0.9235 - val_loss: 0.4981 - val_acc: 0.8796

Epoch 00002: val_acc did not improve from 0.89070
Epoch 3/45
learning rate ---> 0.1 Epoch ---> 102
782/782 [==============================] - 297s 380ms/step - loss: 0.2166 - acc: 0.9241 - val_loss: 0.3381 - val_acc: 0.8995

Epoch 00003: val_acc improved from 0.89070 to 0.89950, saving model to /gdrive/My Drive/CIFAR_10/weights.03-0.90.hdf5
Epoch 4/45
learning rate ---> 0.1 Epoch ---> 103
782/782 [==============================] - 298s 381ms/step - loss: 0.2136 - acc: 0.9255 - val_loss: 0.3725 - val_acc: 0.9003

Epoch 00004: val_

# 150 - 180 Epoch

## Validation Accuracy - 92.19%

In [0]:
model.load_weights("/gdrive/My Drive/CIFAR_10/weights.45-0.92.hdf5")

sgd = keras.optimizers.SGD(lr=0.1 ,  momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

#callbacks


def lr_schedule(epoch):
  
  epoch += 150 
  if epoch <= 120:
    K.set_value(model.optimizer.lr, 0.1)
    print("learning rate --->" , 0.1 , "Epoch --->" , epoch)
    return K.get_value(model.optimizer.lr)
  elif epoch > 120 and epoch <=210:
    K.set_value(model.optimizer.lr, 0.01)
    print("learning rate --->" , 0.01 , "Epoch --->" , epoch)
    return K.get_value(model.optimizer.lr)
  else:
    K.set_value(model.optimizer.lr, 0.001)
    print("learning rate --->" , 0.001 , "Epoch --->" , epoch)
    return K.get_value(model.optimizer.lr)
    
      
  
lr_reducer = LearningRateScheduler(lr_schedule)

filepath = '/gdrive/My Drive/CIFAR_10/weights.{epoch:02d}-{val_acc:.2f}.hdf5'
model_checkpoint = keras.callbacks.ModelCheckpoint(filepath,
                                monitor='val_acc', 
                                verbose=1, 
                                save_best_only=True, 
                                save_weights_only=False, 
                                mode='max', 
                                period=1)


csv_logger = keras.callbacks.CSVLogger(filename =  "fifth__.csv", separator=',', append=False)


datagen = keras.preprocessing.image.ImageDataGenerator(
#     featurewise_std_normalization=True,
#     rotation_range=9 , 
    width_shift_range=0.125,
    height_shift_range=0.125,
    horizontal_flip=True)



datagen.fit(x_train)

model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    callbacks=[lr_reducer , csv_logger , model_checkpoint] ,
#                     steps_per_epoch= len(x_train)/batch_size,
                    epochs=30 , 
                   verbose = 1 , 
                   workers = 4 , 
#                    use_multiprocessing = True , 
                   shuffle = True , 
                   validation_data=(x_test, y_test))

Epoch 1/30
learning rate ---> 0.01 Epoch ---> 150
782/782 [==============================] - 315s 402ms/step - loss: 0.1195 - acc: 0.9584 - val_loss: 0.3483 - val_acc: 0.9168

Epoch 00001: val_acc improved from -inf to 0.91680, saving model to /gdrive/My Drive/CIFAR_10/weights.01-0.92.hdf5
Epoch 2/30
learning rate ---> 0.01 Epoch ---> 151
782/782 [==============================] - 298s 381ms/step - loss: 0.1170 - acc: 0.9584 - val_loss: 0.3473 - val_acc: 0.9173

Epoch 00002: val_acc improved from 0.91680 to 0.91730, saving model to /gdrive/My Drive/CIFAR_10/weights.02-0.92.hdf5
Epoch 3/30
learning rate ---> 0.01 Epoch ---> 152
782/782 [==============================] - 298s 381ms/step - loss: 0.1156 - acc: 0.9589 - val_loss: 0.3588 - val_acc: 0.9165

Epoch 00003: val_acc did not improve from 0.91730
Epoch 4/30
learning rate ---> 0.01 Epoch ---> 153
782/782 [==============================] - 298s 381ms/step - loss: 0.1175 - acc: 0.9584 - val_loss: 0.3484 - val_acc: 0.9159

Epoch 00004: 

# Refresh

In [2]:
#keras library

import keras
import keras.backend as K
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam 
from keras.layers import Input
from keras.callbacks import LearningRateScheduler


# densenet 

def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp
  
  
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg
 




def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output
  
#############################################################################
# Hyperparameters
batch_size = 64
num_classes = 10
epochs = 200
l = 12 #L
num_filter = 36 #k
compression = 0.5
dropout_rate = 0.2
#############################################################################
  
  
  
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]




# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)



#normalizing data

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255





input = Input(shape=(img_height, img_width, channel))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

# Back_Prop_First_Conv2D

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)


Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)



model = Model(inputs=[input], outputs=[output])
model.summary()


Using TensorFlow backend.


170500096/170498071 [==============================] - 27s 0us/step
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 36)   972         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 36)   144         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 36)   0           batch_normalization_1[0][0]      
_________________________________________

# First Top Assignment Accuracy
## Model Evaluate - 92.19%

In [5]:
sgd = keras.optimizers.SGD(lr=0.1 ,  momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

model.load_weights("/gdrive/My Drive/CIFAR_10/weights.17-0.92.hdf5")
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 24s 2ms/step
Test loss: 0.34242540892288087
Test accuracy: 0.9219


# Experimenting to increase accuracy

# Multiple version of models between 180-220. We will select the best.


## /gdrive/My Drive/CIFAR_10/weights.01-0.92.hdf5-92.25%
## /gdrive/My Drive/CIFAR_10/weights.04-0.92.hdf5-92.24%
##/gdrive/My Drive/CIFAR_10/__24__weights.01-0.92.hdf5-92.27%

In [3]:
import keras
import keras.backend as K
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam 
from keras.layers import Input
from keras.callbacks import LearningRateScheduler


# densenet 

def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp
  
  
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg
 




def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output
  
#############################################################################
# Hyperparameters
batch_size = 64
num_classes = 10
epochs = 200
l = 12 #L
num_filter = 36 #k
compression = 0.5
dropout_rate = 0.2
#############################################################################
  
  
  
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]




# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)



#normalizing data

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255





input = Input(shape=(img_height, img_width, channel))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

# Back_Prop_First_Conv2D

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)


Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)



model = Model(inputs=[input], outputs=[output])
model.summary()


170500096/170498071 [==============================] - 37s 0us/step
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 36)   972         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 36)   144         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 36)   0           batch_normalization_1[0][0]      
_________________________________________

# 180 - 220 Epochs --------> Accuracy - 92.92%

In [7]:

model.load_weights("/gdrive/My Drive/CIFAR_10/weights.17-0.92.hdf5")
sgd = keras.optimizers.SGD(lr=0.01 ,decay = 1e-6 ,   momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])



reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', 
                                              factor=0.1, 
                                              patience=1, 
                                              verbose=1, 
                                              mode='auto', 
#                                               min_delta=0.0001, 
#                                               cooldown=0, 
                                              min_lr=0.001)

      
  
# lr_reducer = LearningRateScheduler(lr_schedule)

filepath = '/gdrive/My Drive/CIFAR_10/__24__weights.{epoch:02d}-{val_acc:.2f}.hdf5'
model_checkpoint = keras.callbacks.ModelCheckpoint(filepath,
                                monitor='val_acc', 
                                verbose=1, 
                                save_best_only=True, 
                                save_weights_only=False, 
                                mode='max', 
                                period=1)


csv_logger = keras.callbacks.CSVLogger(filename =  "180-250-epoch__.csv", separator=',', append=False)


datagen = keras.preprocessing.image.ImageDataGenerator(
#     featurewise_std_normalization=True,
#     rotation_range=9 , 
#     width_shift_range=0.125,
#     height_shift_range=0.125,
    horizontal_flip=True)



datagen.fit(x_train)

model.fit_generator(datagen.flow(x_train, y_train, batch_size=128),
                    callbacks=[ csv_logger , model_checkpoint] ,
#                     steps_per_epoch= len(x_train)/batch_size,
                    epochs= 40 , 
                   verbose = 1 , 
                   workers = 4 , 
#                    use_multiprocessing = True , 
                   shuffle = True , 
                   validation_data=(x_test, y_test))




Epoch 1/40
391/391 [==============================] - 276s 705ms/step - loss: 0.0772 - acc: 0.9726 - val_loss: 0.3066 - val_acc: 0.9263

Epoch 00001: val_acc improved from -inf to 0.92630, saving model to /gdrive/My Drive/CIFAR_10/__24__weights.01-0.93.hdf5
Epoch 2/40
391/391 [==============================] - 262s 670ms/step - loss: 0.0745 - acc: 0.9739 - val_loss: 0.3076 - val_acc: 0.9272

Epoch 00002: val_acc improved from 0.92630 to 0.92720, saving model to /gdrive/My Drive/CIFAR_10/__24__weights.02-0.93.hdf5
Epoch 3/40
391/391 [==============================] - 262s 671ms/step - loss: 0.0708 - acc: 0.9752 - val_loss: 0.3173 - val_acc: 0.9258

Epoch 00003: val_acc did not improve from 0.92720
Epoch 4/40
391/391 [==============================] - 263s 673ms/step - loss: 0.0687 - acc: 0.9763 - val_loss: 0.3191 - val_acc: 0.9255

Epoch 00004: val_acc did not improve from 0.92720
Epoch 5/40
391/391 [==============================] - 263s 672ms/step - loss: 0.0702 - acc: 0.9754 - val_lo

# Second Top Assignment Accuracy
## Model Evaluate 92.92%

In [6]:
import keras

sgd = keras.optimizers.SGD(lr=0.01 ,  momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

model.load_weights("/gdrive/My Drive/CIFAR_10/__24__weights.17-0.93.hdf5")
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 25s 2ms/step
Test loss: 0.32899246576428415
Test accuracy: 0.9292


# 10 more Epochs remaining out of 250 to get 94%

#### Remove Augmentation for 1 Epoch.

--------
--------
-------
---------
---------
------------

# Experiment 1 -  9266 - No Improvement

In [8]:

model.load_weights("/gdrive/My Drive/CIFAR_10/__24__weights.17-0.93.hdf5")
sgd = keras.optimizers.SGD(lr=0.01 ,decay = 1e-6 ,   momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])



reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', 
                                              factor=0.1, 
                                              patience=1, 
                                              verbose=1, 
                                              mode='auto', 
#                                               min_delta=0.0001, 
#                                               cooldown=0, 
                                              min_lr=0.001)

      
  
# lr_reducer = LearningRateScheduler(lr_schedule)

filepath = '/gdrive/My Drive/CIFAR_10/__24__weights.{epoch:02d}-{val_acc:.2f}.hdf5'
model_checkpoint = keras.callbacks.ModelCheckpoint(filepath,
                                monitor='val_acc', 
                                verbose=1, 
                                save_best_only=True, 
                                save_weights_only=False, 
                                mode='max', 
                                period=1)


csv_logger = keras.callbacks.CSVLogger(filename =  "180-250-epoch__.csv", separator=',', append=False)


model.fit(x=x_train, 
          y= y_train, 
          callbacks=[ csv_logger , model_checkpoint],
          batch_size=64, 
          epochs=1, 
          verbose=1, 
          validation_split=0.0, 
          validation_data=(x_test, y_test), 
          shuffle=True)
         



Train on 50000 samples, validate on 10000 samples
Epoch 1/1
50000/50000 [==============================] - 306s 6ms/step - loss: 0.0707 - acc: 0.9737 - val_loss: 0.3346 - val_acc: 0.9266

Epoch 00001: val_acc improved from -inf to 0.92660, saving model to /gdrive/My Drive/CIFAR_10/__24__weights.01-0.93.hdf5


# Experiment 2 -  9247 - No Improvement

In [9]:

model.load_weights("/gdrive/My Drive/CIFAR_10/__24__weights.17-0.93.hdf5")
sgd = keras.optimizers.SGD(lr=0.01 ,decay = 1e-6 ,   momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])



reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', 
                                              factor=0.1, 
                                              patience=1, 
                                              verbose=1, 
                                              mode='auto', 
#                                               min_delta=0.0001, 
#                                               cooldown=0, 
                                              min_lr=0.001)

      
  
# lr_reducer = LearningRateScheduler(lr_schedule)

filepath = '/gdrive/My Drive/CIFAR_10/__24__weights.{epoch:02d}-{val_acc:.2f}.hdf5'
model_checkpoint = keras.callbacks.ModelCheckpoint(filepath,
                                monitor='val_acc', 
                                verbose=1, 
                                save_best_only=True, 
                                save_weights_only=False, 
                                mode='max', 
                                period=1)


csv_logger = keras.callbacks.CSVLogger(filename =  "180-250-epoch__.csv", separator=',', append=False)


model.fit(x=x_train, 
          y= y_train, 
          callbacks=[ csv_logger , model_checkpoint],
          batch_size=32, 
          epochs=1, 
          verbose=1, 
          validation_split=0.0, 
          validation_data=(x_test, y_test), 
          shuffle=True)
         



Train on 50000 samples, validate on 10000 samples
Epoch 1/1
50000/50000 [==============================] - 351s 7ms/step - loss: 0.0909 - acc: 0.9676 - val_loss: 0.3337 - val_acc: 0.9247

Epoch 00001: val_acc improved from -inf to 0.92470, saving model to /gdrive/My Drive/CIFAR_10/__24__weights.01-0.92.hdf5


# Experiment 3 - 9275 - No Improvement

In [10]:

model.load_weights("/gdrive/My Drive/CIFAR_10/__24__weights.17-0.93.hdf5")
sgd = keras.optimizers.SGD(lr=0.01 ,decay = 1e-6 ,   momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])



reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', 
                                              factor=0.1, 
                                              patience=1, 
                                              verbose=1, 
                                              mode='auto', 
#                                               min_delta=0.0001, 
#                                               cooldown=0, 
                                              min_lr=0.001)

      
  
# lr_reducer = LearningRateScheduler(lr_schedule)

filepath = '/gdrive/My Drive/CIFAR_10/__24__weights.{epoch:02d}-{val_acc:.2f}.hdf5'
model_checkpoint = keras.callbacks.ModelCheckpoint(filepath,
                                monitor='val_acc', 
                                verbose=1, 
                                save_best_only=True, 
                                save_weights_only=False, 
                                mode='max', 
                                period=1)


csv_logger = keras.callbacks.CSVLogger(filename =  "180-250-epoch__.csv", separator=',', append=False)


model.fit(x=x_train, 
          y= y_train, 
          callbacks=[ csv_logger , model_checkpoint],
          batch_size=128, 
          epochs=1, 
          verbose=1, 
          validation_split=0.0, 
          validation_data=(x_test, y_test), 
          shuffle=True)
         



Train on 50000 samples, validate on 10000 samples
Epoch 1/1
50000/50000 [==============================] - 280s 6ms/step - loss: 0.0623 - acc: 0.9781 - val_loss: 0.3314 - val_acc: 0.9275

Epoch 00001: val_acc improved from -inf to 0.92750, saving model to /gdrive/My Drive/CIFAR_10/__24__weights.01-0.93.hdf5


--------
---------
-----------
---------
----------
------------
--------------
-------------
--------------
---------------
--------------
-------------
-----------------
-------------
--------------------
------------------
-------------------
-----------------
-------------


# Experiment 4 - 9301 - Improvement

# Third Top Assignment Accuracy 
## Model Evaluate - 93%

In [12]:

model.load_weights("/gdrive/My Drive/CIFAR_10/__24__weights.17-0.93.hdf5")
sgd = keras.optimizers.SGD(lr=0.01 ,decay = 1e-6 ,   momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])



reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', 
                                              factor=0.1, 
                                              patience=1, 
                                              verbose=1, 
                                              mode='auto', 
#                                               min_delta=0.0001, 
#                                               cooldown=0, 
                                              min_lr=0.001)

      
  
# lr_reducer = LearningRateScheduler(lr_schedule)

filepath = '/gdrive/My Drive/CIFAR_10/__24__weights.{epoch:02d}-{val_acc:.2f}.hdf5'
model_checkpoint = keras.callbacks.ModelCheckpoint(filepath,
                                monitor='val_acc', 
                                verbose=1, 
                                save_best_only=True, 
                                save_weights_only=False, 
                                mode='max', 
                                period=1)


csv_logger = keras.callbacks.CSVLogger(filename =  "180-250-epoch__.csv", separator=',', append=False)


model.fit(x=x_train, 
          y= y_train, 
          callbacks=[ csv_logger , model_checkpoint],
          batch_size=200, 
          epochs=1, 
          verbose=1, 
          validation_split=0.0, 
          validation_data=(x_test, y_test), 
          shuffle=True)
         



Train on 50000 samples, validate on 10000 samples
Epoch 1/1
50000/50000 [==============================] - 265s 5ms/step - loss: 0.0623 - acc: 0.9784 - val_loss: 0.3289 - val_acc: 0.9301

Epoch 00001: val_acc improved from -inf to 0.93010, saving model to /gdrive/My Drive/CIFAR_10/__24__weights.01-0.93.hdf5


--------------------------------------------
--------------------------------------
------------------------------------------
--------------------------------------
---------------------------------------------
---------------------------------------------
----------------------------------------------
-----------------------------------------------
-----------------------------------------------


# Experimenting to Reach 94%
# Load 9301% model 

In [26]:

model.load_weights("/gdrive/My Drive/CIFAR_10/__24__weights.01-0.93.hdf5")
sgd = keras.optimizers.SGD(lr=0.01 ,decay = 1e-2 ,   momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])



reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', 
                                              factor=0.1, 
                                              patience=1, 
                                              verbose=1, 
                                              mode='auto', 
#                                               min_delta=0.0001, 
#                                               cooldown=0, 
                                              min_lr=0.001)

      
  
# lr_reducer = LearningRateScheduler(lr_schedule)

filepath = '/gdrive/My Drive/CIFAR_10/__24__weights.{epoch:02d}-{val_acc:.2f}.hdf5'
model_checkpoint = keras.callbacks.ModelCheckpoint(filepath,
                                monitor='val_acc', 
                                verbose=1, 
                                save_best_only=True, 
                                save_weights_only=False, 
                                mode='max', 
                                period=1)


csv_logger = keras.callbacks.CSVLogger(filename =  "180-250-epoch__.csv", separator=',', append=False)


model.fit(x=x_train, 
          y= y_train, 
          callbacks=[ reduce_lr , csv_logger , model_checkpoint],
          batch_size=50, 
          epochs=5, 
          verbose=1, 
          validation_split=0.0, 
          validation_data=(x_test, y_test), 
          shuffle=True)
         



Train on 50000 samples, validate on 10000 samples
Epoch 1/5
50000/50000 [==============================] - 318s 6ms/step - loss: 0.0695 - acc: 0.9754 - val_loss: 0.3203 - val_acc: 0.9262

Epoch 00001: val_acc improved from -inf to 0.92620, saving model to /gdrive/My Drive/CIFAR_10/__24__weights.01-0.93.hdf5
Epoch 2/5
50000/50000 [==============================] - 293s 6ms/step - loss: 0.0691 - acc: 0.9752 - val_loss: 0.3223 - val_acc: 0.9270

Epoch 00002: val_acc improved from 0.92620 to 0.92700, saving model to /gdrive/My Drive/CIFAR_10/__24__weights.02-0.93.hdf5
Epoch 3/5
50000/50000 [==============================] - 293s 6ms/step - loss: 0.0683 - acc: 0.9758 - val_loss: 0.3218 - val_acc: 0.9271

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.001.

Epoch 00003: val_acc improved from 0.92700 to 0.92710, saving model to /gdrive/My Drive/CIFAR_10/__24__weights.03-0.93.hdf5
Epoch 4/5
50000/50000 [==============================] - 293s 6ms/step - loss: 0.0703 - acc: 0.9751 - 

# Trying out Stochastic Gradient descent with restarts

[Github LInk](https://gist.github.com/jeremyjordan/5a222e04bb78c242f5763ad40626c452)

In [0]:
from keras.callbacks import Callback
import keras.backend as K
import numpy as np

class SGDRScheduler(Callback):
    '''Cosine annealing learning rate scheduler with periodic restarts.
    # Usage
        ```python
            schedule = SGDRScheduler(min_lr=1e-5,
                                     max_lr=1e-2,
                                     steps_per_epoch=np.ceil(epoch_size/batch_size),
                                     lr_decay=0.9,
                                     cycle_length=5,
                                     mult_factor=1.5)
            model.fit(X_train, Y_train, epochs=100, callbacks=[schedule])
        ```
    # Arguments
        min_lr: The lower bound of the learning rate range for the experiment.
        max_lr: The upper bound of the learning rate range for the experiment.
        steps_per_epoch: Number of mini-batches in the dataset. Calculated as `np.ceil(epoch_size/batch_size)`. 
        lr_decay: Reduce the max_lr after the completion of each cycle.
                  Ex. To reduce the max_lr by 20% after each cycle, set this value to 0.8.
        cycle_length: Initial number of epochs in a cycle.
        mult_factor: Scale epochs_to_restart after each full cycle completion.
    # References
        Blog post: jeremyjordan.me/nn-learning-rate
        Original paper: http://arxiv.org/abs/1608.03983
    '''
    def __init__(self,
                 min_lr,
                 max_lr,
                 steps_per_epoch,
                 lr_decay=1,
                 cycle_length=10,
                 mult_factor=2):

        self.min_lr = min_lr
        self.max_lr = max_lr
        self.lr_decay = lr_decay

        self.batch_since_restart = 0
        self.next_restart = cycle_length

        self.steps_per_epoch = steps_per_epoch

        self.cycle_length = cycle_length
        self.mult_factor = mult_factor

        self.history = {}

    def clr(self):
        '''Calculate the learning rate.'''
        fraction_to_restart = self.batch_since_restart / (self.steps_per_epoch * self.cycle_length)
        lr = self.min_lr + 0.5 * (self.max_lr - self.min_lr) * (1 + np.cos(fraction_to_restart * np.pi))
        return lr

    def on_train_begin(self, logs={}):
        '''Initialize the learning rate to the minimum value at the start of training.'''
        logs = logs or {}
        K.set_value(self.model.optimizer.lr, self.max_lr)

    def on_batch_end(self, batch, logs={}):
        '''Record previous batch statistics and update the learning rate.'''
        logs = logs or {}
        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

        self.batch_since_restart += 1
        K.set_value(self.model.optimizer.lr, self.clr())

    def on_epoch_end(self, epoch, logs={}):
        '''Check for end of current cycle, apply restarts when necessary.'''
        if epoch + 1 == self.next_restart:
            self.batch_since_restart = 0
            self.cycle_length = np.ceil(self.cycle_length * self.mult_factor)
            self.next_restart += self.cycle_length
            self.max_lr *= self.lr_decay
            self.best_weights = self.model.get_weights()

    def on_train_end(self, logs={}):
        '''Set weights to the values from the end of the most recent cycle for best performance.'''
        self.model.set_weights(self.best_weights)
        
        

        


In [31]:

batch_size = 64
epoch_size = 5

model.load_weights("/gdrive/My Drive/CIFAR_10/__24__weights.01-0.93.hdf5")
sgd = keras.optimizers.SGD(lr=0.01 ,decay = 1e-6 ,   momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])



# reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', 
#                                               factor=0.1, 
#                                               patience=1, 
#                                               verbose=1, 
#                                               mode='auto', 
#                                               min_delta=0.0001, 
#                                               cooldown=0, 
#                                               min_lr=0.001)

schedule = SGDRScheduler(min_lr=1e-3,
                         max_lr=1e-2,
                         steps_per_epoch=np.ceil(epoch_size/batch_size),
                         lr_decay=0.6,
                         cycle_length=10,
                         mult_factor=1.5)


      
  

filepath = '/gdrive/My Drive/CIFAR_10/__24__weights.{epoch:02d}-{val_acc:.2f}.hdf5'
model_checkpoint = keras.callbacks.ModelCheckpoint(filepath,
                                monitor='val_acc', 
                                verbose=1, 
                                save_best_only=True, 
                                save_weights_only=False, 
                                mode='max', 
                                period=1)


csv_logger = keras.callbacks.CSVLogger(filename =  "___210-240____epoch__.csv", separator=',', append=False)


model.fit(x=x_train, 
          y= y_train, 
          callbacks=[ schedule , csv_logger , model_checkpoint],
          batch_size=batch_size, 
          epochs=epoch_size, 
          verbose=1, 
          validation_split=0.0, 
          validation_data=(x_test, y_test), 
          shuffle=True)
         



Train on 50000 samples, validate on 10000 samples
Epoch 1/5
  128/50000 [..............................] - ETA: 1:14:01 - loss: 0.0947 - acc: 0.9531

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.897983). Check your callbacks.
  % delta_t_median)


50000/50000 [==============================] - 313s 6ms/step - loss: 0.0665 - acc: 0.9770 - val_loss: 0.3222 - val_acc: 0.9280

Epoch 00001: val_acc improved from -inf to 0.92800, saving model to /gdrive/My Drive/CIFAR_10/__24__weights.01-0.93.hdf5
Epoch 2/5
50000/50000 [==============================] - 300s 6ms/step - loss: 0.0631 - acc: 0.9768 - val_loss: 0.3229 - val_acc: 0.9278

Epoch 00002: val_acc did not improve from 0.92800
Epoch 3/5
50000/50000 [==============================] - 299s 6ms/step - loss: 0.0658 - acc: 0.9761 - val_loss: 0.3229 - val_acc: 0.9279

Epoch 00003: val_acc did not improve from 0.92800
Epoch 4/5
50000/50000 [==============================] - 299s 6ms/step - loss: 0.0657 - acc: 0.9771 - val_loss: 0.3234 - val_acc: 0.9273

Epoch 00004: val_acc did not improve from 0.92800
Epoch 5/5
50000/50000 [==============================] - 299s 6ms/step - loss: 0.0641 - acc: 0.9771 - val_loss: 0.3235 - val_acc: 0.9275

Epoch 00005: val_acc did not improve from 0.92800

AttributeError: ignored

# Try increasing batch size without reducing the learning rate. As mentioned in Google AI paper

[Paper](https://ai.google/research/pubs/pub46644)